In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from ribasim import Model, Node
from ribasim.nodes import basin
from shapely.geometry import Point

In [ ]:
base_dir = Path("c:/bin/ribasim")
model_dir = base_dir / "Crystal_Basin"
data_path = model_dir / "data/input/ACTINFLW.csv"

starttime = "2022-01-01"
endtime = "2023-01-01"
model = Model(
    starttime=starttime,
    endtime=endtime,
    crs="EPSG:4326",
)

# Reservoirs and Public Water Supply
Due to the increase of population and climate change Crystal city has implemented a reservoir upstream to store water for domestic use (See @fig-reservoir).
The reservoir is to help ensure a reliable supply during dry periods.
In this module, the user will update the model to incorporate the reservoir's impact on the whole Crystal Basin.

![Crystal basin with demands and a reservoir](https://s3.deltares.nl/ribasim/doc-image/quickstart/Crystal-basin-with-demands-and-a-reservoir.png){fig-align="left" #fig-reservoir}

## Reservoir
### Add a Basin
This time Basin #3 will function as a reservoir instead of a diversion, meaning it's storage and levels will play an important role for the users (the city and the farmer).
The reservoir has a max. area of $32.3 \text{ km}^2$ and a max. depth of $7 \text{ m}$.
The profile of Basin #3 should change to:

In [ ]:
model.basin.add(
    Node(3, Point(-0.75, -0.5), name="Rsv"),
    [
        basin.Profile(area=[20000000, 32300000], level=[0, 7]),
        basin.State(level=[3.5]),
        basin.Time(time=[starttime, endtime]),
    ],
)

### Adjust the code
Adjust the naming of the Basin in the dictionary mapping and the saving file should be `Crystal_2.1` instead of `*_1.2`.

In [ ]:
toml_path = model_dir / "Crystal_2.1/ribasim.toml"
model.write(toml_path)
cli_path = base_dir / "ribasim_windows/ribasim.exe"

# Dictionary mapping node_ids to names
edge_names = {
    (1, 3): "Main",
    (2, 4): "Minor",
    (3, 6): "IrrA Demand",
    (6, 4): "IrrA Drain",
    (3, 7): "Rsv2Main",
    (7, 4): "Main2Conf",
    (4, 5): "Conf2TRC",
    (5, 8): "TRC2Term",
}

# Dictionary mapping basins (node_ids) to names
node_names = {
    3: "Rsv",
    4: "Conf",
}

df_basin = pd.read_feather(model_dir / "Crystal_2.1/results/basin.arrow")

# Create pivot tables and plot for basin data
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

# Skip the first timestep as it is the initialization step
df_basin_wide = df_basin_wide.iloc[1:]


# Create pivot tables and plot for basin data
df_basin_rsv = df_basin_wide.xs("Rsv", axis=1, level=1, drop_level=False)
df_basin_conf = df_basin_wide.xs("Conf", axis=1, level=1, drop_level=False)


def plot_basin_data(
    ax, ax_twin, df_basin, level_color="b", storage_color="r", title="Basin"
):
    # Plot level data
    for idx, column in enumerate(df_basin["level"].columns):
        ax.plot(
            df_basin.index,
            df_basin["level"][column],
            linestyle="-",
            color=level_color,
            label=f"Level - {column}",
        )

    # Plot storage data
    for idx, column in enumerate(df_basin["storage"].columns):
        ax_twin.plot(
            df_basin.index,
            df_basin["storage"][column],
            linestyle="--",
            color=storage_color,
            label=f"Storage - {column}",
        )

    ax.set_ylabel("Level [m]", color=level_color)
    ax_twin.set_ylabel("Storage [m³]", color=storage_color)

    ax.tick_params(axis="y", labelcolor=level_color)
    ax_twin.tick_params(axis="y", labelcolor=storage_color)

    ax.set_title(title)

    # Combine legends from both axes
    lines, labels = ax.get_legend_handles_labels()
    lines_twin, labels_twin = ax_twin.get_legend_handles_labels()
    ax.legend(lines + lines_twin, labels + labels_twin, loc="upper left")


# Create subplots
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

# Plot Rsv basin data
ax2 = ax1.twinx()  # Secondary y-axis for storage
plot_basin_data(ax1, ax2, df_basin_rsv, title="Reservoir Level and Storage Over Time")

In [ ]:
# Sample data loading and preparation
df_flow = pd.read_feather(model_dir / "Crystal_2.1/results/flow.arrow")
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))
df_flow["name"] = df_flow["edge"].map(edge_names)

# Plot the flow data, interactive plot with Plotly
pivot_flow = df_flow.pivot_table(
    index="time", columns="name", values="flow_rate"
).reset_index()
fig = px.line(
    pivot_flow, x="time", y=pivot_flow.columns[1:], title="Flow Over Time [m3/s]"
)

fig.update_layout(legend_title_text="Edge")
fig.write_html(model_dir / "Crystal_2.1/plot_edges.html")
fig.show()

### Plotting results
@fig-sim7 illustrates the new storage and water level at the reservoir.
As expected, after increasing the profile of basin 3 to mimic the reservoir, its storage capacity increased as well.

![Simulated basin storages and levels](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-storages-and-levels.png){fig-align="left" #fig-sim7}